In [3]:
import pandas as pd 
import numpy as np 

In [5]:
sales_train = pd.read_csv('data/sales_train.csv')
test = pd.read_csv('data/test.csv')
items = pd.read_csv('data/items.csv')
item_categories = pd.read_csv('data/item_categories.csv')
shops = pd.read_csv('data/shops.csv')

In [6]:
sales_train = pd.merge(sales_train,items,how='left',on='item_id')
sales_train = pd.merge(sales_train,item_categories,how='left',on='item_category_id')
sales_train = pd.merge(sales_train,shops,how='left',on ='shop_id')

In [7]:
##break date into day month year 
sales_train['date'] = pd.to_datetime(sales_train['date'],format='%d.%m.%Y')


In [8]:
sales_train['day'] = sales_train['date'].map(lambda x: x.day)
sales_train['month'] = sales_train['date'].map(lambda x: x.month)
sales_train['year'] = sales_train['date'].map(lambda x: x.year)

In [9]:
#potential features 
#price change in percent
#price change relative to cateogory 
#price relative to other stores 
sales_train['sales'] = sales_train['item_price']*sales_train['item_cnt_day']
sales_monthly_train = sales_train.groupby(['year','month','shop_id','item_id'],as_index=False)['sales'].sum()

In [10]:
price_end_month = sales_train[['year','month','shop_id','item_id','item_price']].drop_duplicates(subset=['year','month','shop_id','item_id'],keep='last')

In [11]:
price_end_month.sort_values(['year','month'],inplace=True)
price_end_month['last_price'] = price_end_month.groupby(['shop_id','item_id'])['item_price'].shift(1)


In [12]:
sales_monthly_train = pd.merge(sales_monthly_train,price_end_month,how='left',on=['year','month','shop_id','item_id'])

In [16]:
sales_monthly_train['relative_price'] = sales_monthly_train.groupby(['year','month','item_id'])['last_price'].apply(lambda x: x-x.mean())

In [21]:
sales_monthly_train['relative_price'].fillna(0,inplace=True)

In [29]:
avg_price= sales_monthly_train.groupby(['year','month','item_id'],as_index=False)['last_price'].mean()

In [32]:
avg_price.rename(columns={'last_price':'avg_item_price'},inplace=True)

In [38]:
sales_monthly_train = pd.merge(sales_monthly_train,avg_price,how='left', on=['year','month','item_id'])

In [41]:
# sales_monthly_train.drop(['avg_item_price_x','avg_item_price_y'],axis=1,inplace=True)
sales_monthly_train['last_price'].fillna(sales_monthly_train['avg_item_price'])

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
5                  NaN
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10                 NaN
11                 NaN
12                 NaN
13                 NaN
14                 NaN
15                 NaN
16                 NaN
17                 NaN
18                 NaN
19                 NaN
20                 NaN
21                 NaN
22                 NaN
23                 NaN
24                 NaN
25                 NaN
26                 NaN
27                 NaN
28                 NaN
29                 NaN
              ...     
1609094     549.000000
1609095     299.000000
1609096     399.000000
1609097     449.000000
1609098    1149.000000
1609099    1249.000000
1609100    2199.000000
1609101    1999.000000
1609102    1099.000000
1609103    2399.000000
1609104     499.000000
1609105     699.000000
1609106    

In [49]:
dataset = pd.merge(sales_monthly_train, sales_train,how='left', on =['year','month','shop_id','item_id'])

In [61]:
item_cat_map = sales_train[['item_id','item_category_id']].drop_duplicates()

In [63]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [122]:
dataset= pd.merge(sales_monthly_train,items,how='left',on='item_id')
dataset = pd.merge(dataset,item_categories,how='left',on='item_category_id')
dataset = pd.merge(dataset,shops,how='left',on ='shop_id')

In [123]:
dataset.sort_values(['year','month'],inplace=True)
dataset['last_sales'] = dataset.groupby(['shop_id','item_id'])['sales'].shift(1)


In [124]:
date_map = sales_train[['year','month','date_block_num']].drop_duplicates()
dataset = pd.merge(dataset,date_map,how='left',on=['year','month'])

In [84]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score


In [125]:
dataset=dataset.drop(['item_name','item_category_name','shop_name','item_price'],axis=1)

In [134]:

clf = GradientBoostingRegressor(loss='ls', 
                                n_estimators=250, max_depth=3,
                                learning_rate=.1)



In [105]:
dataset.drop('sales',axis=1).isnull().mean()

year                0.000000
month               0.000000
shop_id             0.000000
item_id             0.000000
last_price          0.263574
relative_price      0.000000
avg_item_price      0.132376
item_category_id    0.000000
last_sales          0.263574
date_block_num      0.000000
dtype: float64

In [113]:
mean_cat_price=dataset.groupby(['year','month','item_category_id'],as_index=False)['last_price'].mean()

In [118]:
mean_cat_price.rename(columns = {'last_price':'mean_cat_price'},inplace=True)

year                0.000000
month               0.000000
item_category_id    0.000000
mean_cat_price      0.060664
dtype: float64

In [126]:
dataset = pd.merge(dataset,mean_cat_price,on=['year','month','item_category_id'],how='left')

In [127]:
dataset['last_price'].fillna(dataset['last_price'])
dataset['avg_item_price'].fillna(dataset['avg_item_price'])

year                0.000000
month               0.000000
shop_id             0.000000
item_id             0.000000
sales               0.000000
last_price          0.263574
relative_price      0.000000
avg_item_price      0.132376
item_category_id    0.000000
last_sales          0.263574
date_block_num      0.000000
mean_cat_price      0.040039
dtype: float64

In [130]:
dataset.dropna(inplace=True,how='any')

In [137]:
#last month sales by item,
#last month sales by category
#shop percentage sale by item
#shop percentage by category 
#last time an item was purchased from a store
#last time an category was purchased from a store 
#relative price of item to store averages 
#

In [138]:
dataset.head()

,year,month,shop_id,item_id,sales,last_price,relative_price,avg_item_price,item_category_id,last_sales,date_block_num,mean_cat_price
63226,2013,2,0,32,2210.0,221.0,-112.427941,333.427941,40,1326.0,1,232.157718
63227,2013,2,0,33,1041.0,347.0,-141.866667,488.866667,37,1041.0,1,484.338264
63228,2013,2,0,35,3458.0,247.0,-137.523810,384.523810,40,247.0,1,232.157718
63233,2013,2,0,51,381.0,127.0,-101.666667,228.666667,57,257.0,1,277.479101
63238,2013,2,0,98,1340.0,268.0,-114.096774,382.096774,40,6700.0,1,232.157718


In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 500,n_jobs=-1)
clf.fit(dataset.drop('sales',axis=1),dataset['sales'])

In [135]:
scores = cross_val_score(clf, dataset.drop('sales',axis=1), dataset['sales'], groups =dataset['date_block_num'], cv=10)

KeyboardInterrupt: 